In [1]:
import pandas as pd
import mysql.connector
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
df = pd.read_csv('attacks_clima.csv', index_col=0)

In [3]:
df.head()

,case_number,year,type,country,area,location,activity,name,age,time,species_,injury,fecha_limpia,fatal,sex,pais,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,wind_950mb,wind_900mb,wind_850mb,wind_800mb,wind_750mb,wind_700mb,wind_650mb,wind_600mb,wind_550mb,wind_500mb,wind_450mb,wind_400mb,wind_350mb,wind_300mb,wind_250mb,wind_200mb,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,2018.06.25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,18h00,White shark,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,USA,4.953125,6.15625,6.8125,6.296875,5.03125,5.296875,5.234375,5.09375,5.078125,6.015625,6.5,6.84375,7.296875,5.265625,4.015625,-0.3125,3.125,3.375,3.6875,3.75,3.609375,3.625,4.046875,4.5625,5.140625,5.671875,6.1875,6.59375,7.0,7.546875,7.90625,7.21875,31.5,97.5,5.078125,-9999.0,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875
1,2018.06.18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonÂ McNeely,11,14h00 -15h00,NaN,Minor injury to left thigh,Jun,N,F,USA,4.953125,6.15625,6.8125,6.296875,5.03125,5.296875,5.234375,5.09375,5.078125,6.015625,6.5,6.84375,7.296875,5.265625,4.015625,-0.3125,3.125,3.375,3.6875,3.75,3.609375,3.625,4.046875,4.5625,5.140625,5.671875,6.1875,6.59375,7.0,7.546875,7.90625,7.21875,31.5,97.5,5.078125,-9999.0,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875
2,2018.06.09,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,48,07h45,NaN,Injury to left lower leg from surfboard skeg,Jun,N,M,USA,4.953125,6.15625,6.8125,6.296875,5.03125,5.296875,5.234375,5.09375,5.078125,6.015625,6.5,6.84375,7.296875,5.265625,4.015625,-0.3125,3.125,3.375,3.6875,3.75,3.609375,3.625,4.046875,4.5625,5.140625,5.671875,6.1875,6.59375,7.0,7.546875,7.90625,7.21875,31.5,97.5,5.078125,-9999.0,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875
3,2018.05.27,2018.0,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,52,NaN,"Lemon shark, 3'",Minor injury to foot. PROVOKED INCIDENT,May,N,M,USA,4.953125,6.15625,6.8125,6.296875,5.03125,5.296875,5.234375,5.09375,5.078125,6.015625,6.5,6.84375,7.296875,5.265625,4.015625,-0.3125,3.125,3.375,3.6875,3.75,3.609375,3.625,4.046875,4.5625,5.140625,5.671875,6.1875,6.59375,7.0,7.546875,7.90625,7.21875,31.5,97.5,5.078125,-9999.0,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875
4,2018.05.26.b,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,15,17h00,"Bull shark, 6'",Lower left leg bitten,May,N,M,USA,4.953125,6.15625,6.8125,6.296875,5.03125,5.296875,5.234375,5.09375,5.078125,6.015625,6.5,6.84375,7.296875,5.265625,4.015625,-0.3125,3.125,3.375,3.6875,3.75,3.609375,3.625,4.046875,4.5625,5.140625,5.671875,6.1875,6.59375,7.0,7.546875,7.90625,7.21875,31.5,97.5,5.078125,-9999.0,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875


In [4]:
df.shape

(4395, 62)

Creamos la BBDD con el nombre de tiburones.

In [5]:
def crear_bbdd(nombre_bbdd):

    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="AlumnaAdalab" 
    )
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)


In [6]:
mydb = crear_bbdd('tiburones')

Conexión realizada con éxito
MySQLCursor: CREATE DATABASE IF NOT EXISTS tiburones;


Cread las tablas de la BBDD:
 - Tabla ataques
 - Tabla clima

In [7]:
def crear_insertar_tabla(nombre_bbdd, contraseña, query):
    
    # nos conectamsos con el servidor usando el conector de sql
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
    # iniciamos el cursor
    mycursor = cnx.cursor()
    
    # intentamos hacer la query
    try: 
        mycursor.execute(query)
        cnx.commit() 
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [8]:
df.dtypes

case_number           object
year                 float64
type                  object
country               object
area                  object
location              object
activity              object
name                  object
age                   object
time                  object
species_              object
injury                object
fecha_limpia          object
fatal                 object
sex                   object
pais                  object
rh_950mb             float64
rh_900mb             float64
rh_850mb             float64
rh_800mb             float64
rh_750mb             float64
rh_700mb             float64
rh_650mb             float64
rh_600mb             float64
rh_550mb             float64
rh_500mb             float64
rh_450mb             float64
rh_400mb             float64
rh_350mb             float64
rh_300mb             float64
rh_250mb             float64
rh_200mb             float64
wind_950mb           float64
wind_900mb           float64
wind_850mb    

In [9]:
tabla_ataques = """
CREATE TABLE IF NOT EXISTS `tiburones`.`ataques` (
  `case_number` INT NOT NULL AUTO_INCREMENT,
  `year` INT NOT NULL,
  `type` CHAR (100) NOT NULL,
  `country` VARCHAR (100) NOT NULL,
  `area` VARCHAR (100) NOT NULL,
  `location` VARCHAR (100) NOT NULL,
  `activity` VARCHAR (100) NOT NULL,
  `name` VARCHAR (100) NOT NULL,
  `age` VARCHAR (3) NOT NULL,
  `species_` VARCHAR (100) NOT NULL,
  `injury` VARCHAR (100) NOT NULL,
  `fecha_limpia` VARCHAR (100) NOT NULL,
  `fatal` VARCHAR (100) NOT NULL,
  `sex` VARCHAR(10) NOT NULL,
  `pais` VARCHAR (100) NOT NULL,
  PRIMARY KEY (`case_number`), )
ENGINE = InnoDB;
"""

In [10]:
crear_insertar_tabla('tiburones', 'AlumnaAdalab', tabla_ataques)

1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ')
ENGINE = InnoDB' at line 17
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ')
ENGINE = InnoDB' at line 17


In [19]:
tabla_clima = '''
CREATE TABLE IF NOT EXISTS `tiburones`.`clima` (
  `index` FLOAT NOT NULL AUTO_INCREMENT,
  `rh_950mb` FLOAT NOT NULL,
  `rh_900mb` FLOAT NOT NULL,
  `rh_850mb` FLOAT NOT NULL,
  `rh_800mb` FLOAT NOT NULL,
  `rh_750mb` FLOAT NOT NULL,
  `rh_700mb` FLOAT NOT NULL,
  `rh_650mb` FLOAT NOT NULL,
  `rh_600mb` FLOAT NOT NULL,
  `rh_550mb` FLOAT NOT NULL,
  `rh_500mb` DATETIME,
  `rh_450mb` FLOAT NOT NULL,
  `rh_400mb` FLOAT NOT NULL,
  `rh_350mb` FLOAT NOT NULL,
  `rh_300mb` FLOAT NOT NULL,
  `rh_250mb` FLOAT NOT NULL,
  `rh_200mb` FLOAT NOT NULL,
  `wind_950mb` FLOAT NOT NULL,
  `wind_900mb` FLOAT NOT NULL,
  `wind_850mb` FLOAT NOT NULL,
  `wind_800mb` FLOAT NOT NULL,
  `wind_750mb` FLOAT NOT NULL,
  `wind_700mb` FLOAT NOT NULL,
  `wind_650mb` FLOAT NOT NULL,
  `wind_600mb` FLOAT NOT NULL,
  `wind_550mb` FLOAT NOT NULL,
  `wind_500mb` FLOAT NOT NULL,
  `wind_450mb` FLOAT NOT NULL,
  `wind_400mb` FLOAT NOT NULL,
  `wind_350mb` FLOAT NOT NULL,
  `wind_300mb` FLOAT NOT NULL,
  `wind_250mb` FLOAT NOT NULL,
  `wind_2000mb` FLOAT NOT NULL,
  `timepoint` FLOAT NOT NULL, 
  `cloudcover` FLOAT NOT NULL, 
  `highcloud` FLOAT NOT NULL, 
  `midcloud` FLOAT NOT NULL, 
  `lowcloud` FLOAT NOT NULL, 
  `temp2m` FLOAT NOT NULL, 
  `lifted_index` FLOAT NOT NULL, 
  `rh2m` FLOAT NOT NULL, 
  `msl_pressure` FLOAT NOT NULL, 
  `prec_amount` FLOAT NOT NULL, 
  `snow_depth` FLOAT NOT NULL,
  `wind10m.direction` FLOAT NOT NULL,
  `wind10m.speed` FLOAT NOT NULL,

  
  PRIMARY KEY (`index`))
ENGINE = InnoDB;
'''

In [20]:
crear_insertar_tabla('tiburones', 'AlumnaAdalab', tabla_clima)